In [94]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch import optim
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.preprocessing import StandardScaler

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/traintest/train.csv
/kaggle/input/traintest/test.csv


In [95]:
data_train = pd.read_csv("/kaggle/input/traintest/train.csv")
data_test = pd.read_csv("/kaggle/input/traintest/test.csv")
data_train = data_train.iloc[:-1]
data_train = data_train.fillna(0)
data_test = data_test.fillna(0)
y_train = data_train['SalePrice']
data_train = data_train.drop(columns=['SalePrice'])




print(data_train.isna().sum())  # Для обучающего датасета
print(data_test.isna().sum())   # Для тестового датасета

# Проверка после создания dummies

cat_features_mask = (data_train.dtypes == "object").values
cat_features_mask_test = (data_test.dtypes == "object").values

data_train_cat = data_train[data_train.columns[cat_features_mask]]
data_test_cat = data_test[data_test.columns[cat_features_mask_test]]







combined = pd.concat([data_train_cat, data_test_cat], axis=0)
combined_dummies = pd.get_dummies(combined, drop_first=True)
train_size = len(data_train_cat)
dtraincd = combined_dummies[:train_size]  
dtestcd = combined_dummies[train_size:]
 
dtraincd = dtraincd.replace({True: 1, False: 0})
dtestcd = dtestcd.replace({True: 1, False: 0})

train_concat = pd.concat([data_train[data_train.columns[~cat_features_mask]], dtraincd], axis=1)
test_concat = pd.concat([data_test[data_test.columns[~cat_features_mask_test]], dtestcd], axis=1)

X_train_tensor = torch.tensor(train_concat.values, dtype=torch.float32)
X_test_tensor = torch.tensor(test_concat.values, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor)

print(train_concat.shape, test_concat.shape)

print(dtraincd.shape, dtestcd.shape)
print(train_concat.isna().sum().sum())  # Для обработанных категориальных признаков (обучающий)
print(test_concat.isna().sum().sum())  



combined = pd.concat([data_train_cat, data_test_cat], axis=0)
combined_dummies = pd.get_dummies(combined, drop_first=True)
train_size = len(data_train_cat)
dtraincd = combined_dummies[:train_size]  
dtestcd = combined_dummies[train_size:]

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 80, dtype: int64
Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 80, dtype: int64


/tmp/ipykernel_30/2733095456.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dtraincd = dtraincd.replace({True: 1, False: 0})
/tmp/ipykernel_30/2733095456.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dtestcd = dtestcd.replace({True: 1, False: 0})


(1459, 268) (1459, 268)
(1459, 231) (1459, 231)
0
0


Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 80, dtype: int64
Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 80, dtype: int64


/tmp/ipykernel_30/979139014.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dtraincd = dtraincd.replace({True: 1, False: 0})
/tmp/ipykernel_30/979139014.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dtestcd = dtestcd.replace({True: 1, False: 0})


(1459, 268) (1459, 268)
(1459, 231) (1459, 231)
0
0


In [96]:
trainLoader = DataLoader(train_dataset, batch_size=64)
testLoader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [97]:
model = nn.Sequential(
    nn.Linear(268, 512),
    nn.ReLU(),
    nn.Linear(in_features=512, out_features=256),
    nn.ReLU(),
    nn.Linear(in_features=256, out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=64),
    nn.ReLU(),
    nn.Linear(in_features=64, out_features=1),
    
)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)






In [98]:
def run(model, dataloader, loss_function, optimizer=None):
    # set the model to evaluation or training mode
    if optimizer is None:
        model.eval()
    else:
        model.train()

    total_loss = 0

    for X, y in dataloader:
        # Ensure that the input data has the correct shape
          # Debugging line to check shape

        # Zero the gradients before backward pass
        optimizer.zero_grad()  # Clear previous gradients

        # compute prediction
        pred = model(X)

        # compute loss
        loss = loss_function(pred, y)

        # save loss
        total_loss += loss.item()

        if optimizer is not None:
            # compute gradients
            loss.backward()

            

            # do optimizer step
            optimizer.step()

    return model.parameters()

NUM_EPOCHS = 20
for i in range(NUM_EPOCHS):
    run(model, trainLoader, loss_function, optimizer)
print("Model parameters after training:")


# Test phase
y_test_pred = []
model.eval()  
with torch.no_grad(): 
    for X_batch in testLoader:
        X_batch = X_batch[0]
        pred = model(X_batch) 
        y_test_pred.append(pred)

y_test_pred = torch.cat(y_test_pred, dim=0).squeeze().numpy()
output = pd.DataFrame({'Id': data_test.Id, 'SalePrice': y_test_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
y_test_pred


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([51])) that is different to the input size (torch.Size([51, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Model parameters after training:
Your submission was successfully saved!


array([163046.28, 183614.36, 169095.2 , ..., 171990.89, 147845.39,
       184137.84], dtype=float32)